In [1]:
import numpy as np
import pandas as pd

In [350]:
path = 'train_words.txt'
words = pd.read_csv(path).sample(frac=0.1)##extracting 10 percent of total size in RANDOM order
words.dropna(inplace=True)

In [351]:
words.shape

(37010, 1)

In [326]:
train_w = list(words.iloc[:,0].values)

In [364]:
train_w[:10]

['suburbanities',
 'chloroplastid',
 'glaikit',
 'ropable',
 'vesicate',
 'odontography',
 'sedulous',
 'anargyros',
 'hypergrammatical',
 'benchboard']

# Generating text attributes

In [309]:
n = 1000
uni_l1 = np.zeros(shape=(n,36), dtype=np.int8)
uni_l2 = np.zeros(shape=(n,72), dtype=np.int8)
uni_l3 = np.zeros(shape=(n,36*3), dtype=np.int8)
uni_l4 = np.zeros(shape=(n,36*4), dtype=np.int8)
uni_l5 = np.zeros(shape=(n,36*5), dtype=np.int8)
bin_l2 = np.zeros(shape=(n,50), dtype=np.int8)
tri_l2 = np.zeros(shape=(n,20), dtype=np.int8)

In [175]:
def unary(words, k=2):
#     word = word.lower()
    freq = np.zeros(shape=(len(words), 36*k), dtype=np.int8)
    for iw,word in enumerate(words):
        parts = []; l = len(word);
        if (k==1):parts=[word]
        elif (k>=l):
            parts = list(word) 
            parts=[]*(k-l)+parts 
        else:
            i = 0; div = int(l/k)
            parts = [word[i:i+div] for i in range(0, div*(k-1), div)]
            parts.append(word[div*(k-1):])
        for p,part in enumerate(parts):
            for num in range(97, 123):
                if chr(num) in part:
                    freq[iw,p*36 + num-97] = 1
            for num in range(48, 58):
                if chr(num) in part:
                    freq[iw,p*36 + 26 + num-48] = 1                
    return freq

In [401]:
def binary_2(words, top_n=50):
    first_h = pd.DataFrame(data=np.zeros((len(words),1)), 
                                     columns=['ti'])
    second_h = pd.DataFrame(data=np.zeros((len(words),1)), 
                                         columns=['ti'])
    parts=[x[:] for x in [[None]]*len(words)]
    for iw, word in enumerate(words):
        if(len(word) <= 4):
            parts[iw] = [word]
        else:
            div = int(len(word)/2)
            parts[iw]=[word[:div]] + [word[div:]]
    parts = pd.DataFrame(parts) 
    parts.fillna('', inplace=True)
    for col in parts:
        for iw,word in enumerate(parts[col]):
    #         print(iw, word)
            bgs=[word[i-1:i+1] for i in range(1, len(word))]
            for bg in bgs:
                if (col==0):first_h.loc[iw,bg] = 1
                else:     second_h.loc[iw,bg] = 1
    first_h.fillna(0, inplace=True)
    second_h.fillna(0, inplace=True)
    top_1 = first_h.mean(axis=0).sort_values(ascending=False)[:top_n]
    top_2 = second_h.mean(axis=0).sort_values(ascending=False)[:top_n]
#     print(top_1, top_2)
    return np.hstack((first_h[top_1.index], second_h[top_2.index])).astype(int)

In [400]:
def trinary_2(words, top_n = 20):
    first_h = pd.DataFrame(data=np.zeros((len(words),1)), 
                                     columns=['ti'])
    second_h = pd.DataFrame(data=np.zeros((len(words),1)), 
                                         columns=['ti'])
    parts=[x[:] for x in [[None]]*len(words)]
    for iw, word in enumerate(words):
        if(len(word) <= 4):
            parts[iw] = [word]
        else:
            div = int(len(word)/2)
            parts[iw]=[word[:div]] + [word[div:]]
    parts = pd.DataFrame(parts) 
    parts.fillna('', inplace=True)
    for col in parts:
        for iw,word in enumerate(parts[col]):
#             print(iw, word)
            try:
                bgs=[word[i-2:i+1] for i in range(2, len(word))]
            except(TypeError):print(word, type(word))
            for bg in bgs:
                if (col==0):first_h.loc[iw,bg] = 1
                else:     second_h.loc[iw,bg] = 1
    first_h.fillna(0, inplace=True)
    second_h.fillna(0, inplace=True)
    top_1 = first_h.mean(axis=0).sort_values(ascending=False)[:top_n]
    top_2 = second_h.mean(axis=0).sort_values(ascending=False)[:top_n]
#     print(top_1, top_2)
    return np.hstack((first_h[top_1.index], second_h[top_2.index])).astype(int)

In [405]:
tri_l2 = trinary_2(train_w[:n])

In [406]:
tri_l2.shape

(1000, 40)

In [407]:
bin_l2 = binary_2(train_w[:n])

In [408]:
bin_l2.shape

(1000, 100)

In [409]:
uni_l5 = unary(train_w[:n], k=5)
uni_l4 = unary(train_w[:n], k=4)
uni_l3 = unary(train_w[:n], k=3)
uni_l2 = unary(train_w[:n], k=2)
uni_l1 = unary(train_w[:n], k=1)

In [410]:
for m in map(np.shape, [uni_l1, uni_l2, uni_l3, uni_l4, uni_l5]):
    print(m)

(1000, 36)
(1000, 72)
(1000, 108)
(1000, 144)
(1000, 180)


In [411]:
text_att = np.hstack((uni_l1, uni_l2, uni_l3, uni_l4, uni_l5, bin_l2, tri_l2))

In [412]:
text_att.shape

(1000, 680)

# Network Architecture

In [187]:
import tensorflow as tf
conv = tf.layers.conv2d
maxp = tf.layers.max_pooling2d
batn = tf.layers.batch_normalization
Relu = tf.nn.relu
# mxout = tf.contrib.layers.maxout

In [188]:
tf.contrib.layers.maxout

AttributeError: module 'tensorflow.contrib.layers' has no attribute 'maxout'

In [203]:
tf.reset_default_graph()
input = tf.placeholder(shape=[None, 32, 100, 1], dtype=tf.float32)
net = conv(input, kernel_size=3, filters=64, strides=1, padding='same',activation=Relu, name='conv_64.1')
print(net.name + "\t" + str(net.shape))
net = batn(net, name='bn1')
print(net.name + "\t" + str(net.shape))
print('Maxout')
net = conv(net, kernel_size=3, filters=64, strides=1, padding='same',activation=Relu, name='conv_64.2')
print(net.name + "\t" + str(net.shape))
net = batn(net, name='bn2')
print(net.name + "\t" + str(net.shape))
print('Maxout')
net = conv(net, kernel_size=3, filters=64, strides=1, padding='same',activation=Relu, name='conv_64.3')
print(net.name + "\t" + str(net.shape))
net = batn(net, name='bn3')
print(net.name + "\t" + str(net.shape))
print('Maxout')
net = maxp(net, pool_size=2, strides=2, padding='same', name='MPool_100_50')
print(net.name + "\t" + str(net.shape))
print()

net = conv(net, kernel_size=3, filters=128, strides=1, padding='same',activation=Relu, name='conv_128.1')
print(net.name + "\t" + str(net.shape))
net = batn(net, name='bn4')
print(net.name + "\t" + str(net.shape))
print('Maxout')
net = conv(net, kernel_size=3, filters=128, strides=1, padding='same',activation=Relu, name='conv_128.2')
print(net.name + "\t" + str(net.shape))
net = batn(net, name='bn5')
print(net.name + "\t" + str(net.shape))
print('Maxout')
net = maxp(net, pool_size=2, strides=2, padding='same', name='MPool_50_25')
print(net.name + "\t" + str(net.shape))
print()

net = conv(net, kernel_size=3, filters=256, strides=1, padding='same',activation=Relu, name='conv_256.1')
print(net.name + "\t" + str(net.shape))
net = batn(net, name='bn6')
print(net.name + "\t" + str(net.shape))
print('Maxout')
net = conv(net, kernel_size=3, filters=256, strides=1, padding='same',activation=Relu, name='conv_256.2')
print(net.name + "\t" + str(net.shape))
net = batn(net, name='bn7')
print(net.name + "\t" + str(net.shape))
print('Maxout')
net = maxp(net, pool_size=2, strides=2, padding='same', name='MPool_25_13')
print(net.name + "\t" + str(net.shape))
print()

net = conv(net, kernel_size=3, filters=512, strides=1, padding='same',activation=Relu, name='conv_512.1')
print(net.name + "\t" + str(net.shape))
net = batn(net, name='bn8')
print(net.name + "\t" + str(net.shape))
print('Maxout')
net = conv(net, kernel_size=3, filters=512, strides=1, padding='same',activation=Relu, name='conv_512.2')
print(net.name + "\t" + str(net.shape))
net = batn(net, name='bn9')
print(net.name + "\t" + str(net.shape))
print('Maxout')
print()



conv_64.1/Relu:0	(?, 32, 100, 64)
bn1/batchnorm/add_1:0	(?, 32, 100, 64)
Maxout
conv_64.2/Relu:0	(?, 32, 100, 64)
bn2/batchnorm/add_1:0	(?, 32, 100, 64)
Maxout
conv_64.3/Relu:0	(?, 32, 100, 64)
bn3/batchnorm/add_1:0	(?, 32, 100, 64)
Maxout
MPool_100_50/MaxPool:0	(?, 16, 50, 64)

conv_128.1/Relu:0	(?, 16, 50, 128)
bn4/batchnorm/add_1:0	(?, 16, 50, 128)
Maxout
conv_128.2/Relu:0	(?, 16, 50, 128)
bn5/batchnorm/add_1:0	(?, 16, 50, 128)
Maxout
MPool_50_25/MaxPool:0	(?, 8, 25, 128)

conv_256.1/Relu:0	(?, 8, 25, 256)
bn6/batchnorm/add_1:0	(?, 8, 25, 256)
Maxout
conv_256.2/Relu:0	(?, 8, 25, 256)
bn7/batchnorm/add_1:0	(?, 8, 25, 256)
Maxout
MPool_25_13/MaxPool:0	(?, 4, 13, 256)

conv_512.1/Relu:0	(?, 4, 13, 512)
bn8/batchnorm/add_1:0	(?, 4, 13, 512)
Maxout
conv_512.2/Relu:0	(?, 4, 13, 512)
bn9/batchnorm/add_1:0	(?, 4, 13, 512)
Maxout

